<a href="https://colab.research.google.com/github/dany-gaga/ligand_prediction_for_ORs/blob/notebook/model_and_prediction_for_ORs_using_coconut_database.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# run this code to mount Google Drive
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [ ]:
%cd /content/gdrive/MyDrive/ml_model_for_Serge

/content/gdrive/MyDrive/ml_model_for_Serge


In [ ]:
!pip install rdkit

In [ ]:
import pandas as pd
import numpy as np
import rdkit
from rdkit.Chem import PandasTools
from rdkit.Chem.Draw import IPythonConsole
IPythonConsole.ipython_useSVG=True  #< set this to False if you want PNGs instead of SVGs
from functools import reduce
from sklearn.model_selection import train_test_split
import tensorflow  as tf
import keras
from tensorflow.keras import optimizers
from tensorflow.keras import regularizers
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.utils import CustomObjectScope
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Model, save_model, load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.layers import Input,Dropout, Embedding, Concatenate,Conv1D,GlobalAveragePooling1D, Dense
from keras.models import Model, load_model
import matplotlib.pyplot as plt
tf.random.set_seed(42) # for constant random

Reviewed

In [ ]:
!wget https://www.bindingdb.org/bind/downloads/BindingDB_All_202403_tsv.zip

--2024-03-25 14:45:01--  https://www.bindingdb.org/bind/downloads/BindingDB_All_202403_tsv.zip
Resolving www.bindingdb.org (www.bindingdb.org)... 137.110.139.247
Connecting to www.bindingdb.org (www.bindingdb.org)|137.110.139.247|:443... connected.
HTTP request sent, awaiting response... 200 200
Length: 481248218 (459M) [application/zip]
Saving to: ‘BindingDB_All_202403_tsv.zip’

BindingDB_All_20240 100%[===================>] 458.95M  7.66MB/s    in 51s     

2024-03-25 14:45:53 (9.08 MB/s) - ‘BindingDB_All_202403_tsv.zip’ saved [481248218/481248218]



In [ ]:
columns=[ 'Ligand SMILES', 'Ki (nM)','BindingDB Target Chain Sequence']
df = pd.read_csv("/content/gdrive/MyDrive/ml_model_for_Serge/BindingDB_All_202403_tsv.zip",delimiter="\t",usecols=columns)
df = df[df["Ki (nM)"].notna()]

<ipython-input-8-0f69b816c5dd>:2: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/content/gdrive/MyDrive/ml_model_for_Serge/BindingDB_All_202403_tsv.zip",delimiter="\t",usecols=columns)


In [ ]:
df


,Ligand SMILES,Ki (nM),BindingDB Target Chain Sequence
0,O[C@@H]1[C@@H](O)[C@@H](Cc2ccccc2)N(CCCCCC(O)=...,0.24,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMSLPGRWKPKM...
1,O[C@@H]1[C@@H](O)[C@@H](Cc2ccccc2)N(C\C=C\c2cn...,0.25,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMSLPGRWKPKM...
2,O[C@@H]1[C@@H](O)[C@@H](Cc2ccccc2)N(CC2CC2)C(=...,0.41,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMSLPGRWKPKM...
3,OCCCCCCN1[C@H](Cc2ccccc2)[C@H](O)[C@@H](O)[C@@...,0.8,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMSLPGRWKPKM...
4,OCCCCCN1[C@H](Cc2ccccc2)[C@H](O)[C@@H](O)[C@@H...,0.99,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMSLPGRWKPKM...
...,...,...,...
2836924,COc1cccc2c(ccc(O)c12)-c1ccc2cccc(OC)c2c1O,6600,MGETLGDSPIDPESDSFTDTLSANISQEMTMVDTEMPFWPTNFGIS...
2836925,CCc1ccc(CCOc2ccc(CC3SC(=O)NC3=O)cc2)nc1,600,MGETLGDSPIDPESDSFTDTLSANISQEMTMVDTEMPFWPTNFGIS...
2836926,CN(CCOc1ccc(C[C@H](Nc2ccccc2C(=O)c2ccccc2)C(O)...,2.0,MGETLGDSPIDPESDSFTDTLSANISQEMTMVDTEMPFWPTNFGIS...
2836927,COc1cccc2ccc(c(O)c12)-c1cc(-c2ccc3cccc(OC)c3c2...,18800,MEQPQEEAPEVREEEEKEEVAEAEGAPELNGGPQHALPSSSYTDLS...


In [ ]:
#process the data
df["Ki (nM)"]=df["Ki (nM)"].astype(str).apply(lambda x: x.replace('>', '',))
df["Ki (nM)"]=df["Ki (nM)"].astype(str).apply(lambda x: x.replace('<', '',)).astype(float)
df['Pki'] = np.log(df["Ki (nM)"].div(1e9))*-1
df=df[(df["Pki"]<24) & (df["Pki"]>2)]
df["protein_length"] = df["BindingDB Target Chain Sequence"].astype(str).apply(lambda x:len(x))
df["ligand_length"] = df["Ligand SMILES"].astype(str).apply(lambda x:len(x))
df = df[(df["protein_length"]<600) & (df["protein_length"]>200)]

In [ ]:
df

,Ligand SMILES,Ki (nM),BindingDB Target Chain Sequence,Pki,protein_length,ligand_length
198,CN(Cc1ccc(s1)C(=O)N[C@@H](CC(O)=O)C(=O)CSCc1cc...,90.0,MENTENSVDSKSIKNLEPKIIHGSESMDSGISLDNSYKMDYPEMGL...,16.223456,277,73
199,OC(=O)C[C@H](NC(=O)c1ccc(CNS(=O)(=O)c2ccc(O)c(...,160.0,MADKVLKEKRKLFIRSMGEGTINGLLDELLQTRVLNKEEMEKVKRE...,15.648092,404,63
200,OC(=O)C[C@H](NC(=O)c1ccc(CNS(=O)(=O)c2ccc(O)c(...,3900.0,MADKVLKEKRKLFIRSMGEGTINGLLDELLQTRVLNKEEMEKVKRE...,12.454534,404,63
201,CN(Cc1ccc(s1)C(=O)N[C@@H](CC(O)=O)C(=O)CSCc1cc...,12000.0,MADKVLKEKRKLFIRSMGEGTINGLLDELLQTRVLNKEEMEKVKRE...,11.330604,404,73
202,OC(=O)C[C@H](NC(=O)c1ccc(CNS(=O)(=O)c2ccc(O)c(...,1500.0,MADKVLKEKRKLFIRSMGEGTINGLLDELLQTRVLNKEEMEKVKRE...,13.410045,404,78
...,...,...,...,...,...,...
2836924,COc1cccc2c(ccc(O)c12)-c1ccc2cccc(OC)c2c1O,6600.0,MGETLGDSPIDPESDSFTDTLSANISQEMTMVDTEMPFWPTNFGIS...,11.928441,505,41
2836925,CCc1ccc(CCOc2ccc(CC3SC(=O)NC3=O)cc2)nc1,600.0,MGETLGDSPIDPESDSFTDTLSANISQEMTMVDTEMPFWPTNFGIS...,14.326336,505,39
2836926,CN(CCOc1ccc(C[C@H](Nc2ccccc2C(=O)c2ccccc2)C(O)...,2.0,MGETLGDSPIDPESDSFTDTLSANISQEMTMVDTEMPFWPTNFGIS...,20.030119,505,61
2836927,COc1cccc2ccc(c(O)c12)-c1cc(-c2ccc3cccc(OC)c3c2...,18800.0,MEQPQEEAPEVREEEEKEEVAEAEGAPELNGGPQHALPSSSYTDLS...,10.881654,441,63


In [ ]:
df.to_csv('for_test_short.csv')

**Novel start**

In [ ]:
# 600 for length as the protein are no longer than 500
df = pd.read_csv('for_test_short.csv')

In [ ]:
#split data
df2=df.sample(n=427269,replace=False)
X =df2[["Ligand SMILES","BindingDB Target Chain Sequence"]]
y = df2[["Pki"]]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
#create a list to be used in NLP
#traning list
smiles_train =list(X_train["Ligand SMILES"])
protein_train=list(X_train["BindingDB Target Chain Sequence"])
label_train=list(y_train['Pki'])

In [ ]:
#test list
smiles_test =list(X_test["Ligand SMILES"])
protein_test=list(X_test["BindingDB Target Chain Sequence"])
label_test=list(y_test['Pki'])

In [ ]:
# smiles train
tokenizer_smiles = Tokenizer(char_level= True)
tokenizer_smiles.fit_on_texts(smiles_train)
word_index_smiles = tokenizer_smiles.word_index
vocab_size_smiles = len(word_index_smiles)
train_sequence_smiles = tokenizer_smiles.texts_to_sequences(smiles_train)
train_padded_smiles = pad_sequences(train_sequence_smiles,truncating="post",
                                   padding="post",maxlen=85)
# test smiles
test_sequence_smiles = tokenizer_smiles.texts_to_sequences(smiles_test)
test_padded_smiles = pad_sequences(test_sequence_smiles,truncating="post",
                                   padding="post",maxlen=85)

In [ ]:
# protein data
#train protins
protein_train
tokenizer_protein = Tokenizer(char_level= True)
tokenizer_protein.fit_on_texts(protein_train)
word_index_protein = tokenizer_protein.word_index
vocab_size_protein= len(word_index_protein)
train_sequence_protein = tokenizer_protein.texts_to_sequences(protein_train)
train_padded_protein = pad_sequences(train_sequence_protein,truncating="post",
                                   padding="post",maxlen=600)
#test  proteins
test_sequence_protein = tokenizer_protein.texts_to_sequences(protein_test)
test_padded_protein = pad_sequences(test_sequence_protein,truncating="post",
                                   padding="post", maxlen = 600)

In [ ]:
#convert to array
# smiles
train_smiles_array = np.array(train_padded_smiles)
test_smiles_array = np.array(test_padded_smiles)
# proteins
train_protein_array = np.array(train_padded_protein)
test_protein_array = np.array(test_padded_protein)
# bindig afinity
train_label_array = np.array(label_train,dtype ="float32")
test_label_array = np.array(label_test,dtype ="float32")
#input layers
smile_input =Input((85,))
protein_input = Input((600,))

In [ ]:
def cindex_score(y_true, y_pred):
    g = tf.subtract(tf.expand_dims(y_pred, -1), y_pred)
    g = tf.cast(g == 0.0, tf.float32) * 0.5 + tf.cast(g > 0.0, tf.float32)
    f = tf.subtract(tf.expand_dims(y_true, -1), y_true) > 0.0
    f = tf.compat.v1.matrix_band_part(tf.cast(f, tf.float32), -1, 0)
    g = tf.reduce_sum(tf.multiply(g, f))
    f = tf.reduce_sum(f)
    return tf.where(tf.equal(g, 0), 0.0, g/f)

In [ ]:
def cnn_block(vocab_size,length,input_shape):
    """
    create embeding and cnn layers for either protein or smiles
    """
    embed = Embedding(vocab_size+1, 128, input_length=length)(input_shape)
    conv1 = Conv1D(32,4, activation="relu")(embed)
    conv2 = Conv1D(64,6, activation="relu")(conv1)
    conv3 = Conv1D(96,8, activation="relu")(conv2)
    max_pool = GlobalAveragePooling1D()(conv3)
    return max_pool

In [ ]:
# concatenate the two layers
conct_layers = Concatenate()([cnn_block(vocab_size_smiles,85,smile_input),#smiles layer
                              cnn_block(vocab_size_protein,85,protein_input)]#protein layer
                     )

In [ ]:
# output network
output=Dense(1024,activation="relu")(conct_layers)
output = Dropout(0.1)(output)
output=Dense(1024,activation="relu")(output)
output = Dropout(0.1)(output)
output=Dense(512,activation="relu")(output)
output = Dense(1)(output)

In [ ]:
# model difinition and optmization
model=Model([protein_input, smile_input],output)
# compile the model with the custom metric
with CustomObjectScope({'cindex_score': cindex_score}):
    optimizer = optimizers.Adam(learning_rate=0.001)
    model.compile(loss='mse', optimizer=optimizer, metrics=['mse', cindex_score])

In [ ]:
#train and evalution
early_stop =EarlyStopping(min_delta=0.001,patience=5,restore_best_weights=True,)
history=model.fit([train_protein_array,train_smiles_array],train_label_array,
                 epochs=100,validation_data=([test_protein_array,test_smiles_array],
                                                           test_label_array),callbacks=[early_stop],verbose=1)

In [ ]:
# save the model
model.save('my_model_600.keras')

In [ ]:
hist = pd.DataFrame(history.history)
hist.to_csv("./model_eval.csv")
plt.plot(history.history['mse'])
plt.plot(history.history['val_mse'])
plt.title('model mse')
plt.ylabel('mse')
plt.xlabel('epoch')
plt.legend(['train','test'], loc='upper left')
plt.savefig('./mse.png')   # save the figure to file

In [ ]:
## loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train','test'], loc='upper left')
plt.savefig('./loss.png')


Predictions

In [ ]:
!wget https://coconut.naturalproducts.net/download/sdftar

In [ ]:
# Convert to a Pandas DataFrame
dfcoco = PandasTools.LoadSDF('COCONUT_2022_01_2D.SDF')


In [ ]:
req_columns =['coconut_id','name','SMILES','molecular_weight','molecular_formula','citationDOI','alogp','topoPSA']
cocodata =pd.read_csv("./coconut_db.csv",usecols=req_columns,index_col='coconut_id')
cocodata.sample(n=600,replace=False).to_csv("test_coconut_data.csv")
cocodata =pd.read_csv("test_coconut_data.csv")
cocodata.shape

In [ ]:
def protein_seq(file):
    """
    process the protein sequence and return the sequence in one line
    """
    protein=''
    with open(file) as fh:
        for line in fh:
            if line.startswith(">"):
                pass
            else:protein+=line.replace("\n","")
    return protein

In [ ]:
n=cocodata.shape[0]
list_or1=[protein_seq("or1.fa")]*n
list_or2=[protein_seq("or2.fa")]*n
list_or3=[protein_seq("or3.fa")]*n
list_or4=[protein_seq("or4.fa")]*n
smiles_list=list(cocodata["SMILES"])


In [ ]:
#load  model
model = keras.models.load_model('my_model_600.keras')

In [ ]:
def predictpkd(protein,smiles):
    """
    process the protein and smiles and make predictions using the trained model
    """
    tokenizer_protein = Tokenizer(char_level= True)
    tokenizer_protein.fit_on_texts(protein)
    protein=tokenizer_protein.texts_to_sequences(protein)
    protein_padded = pad_sequences(protein,truncating="post",
                                   padding="post",maxlen=600)
    tokenizer_smiles = Tokenizer(char_level= True)
    tokenizer_smiles.fit_on_texts(smiles)
    smiles=tokenizer_smiles.texts_to_sequences(smiles)
    smiles_padded = pad_sequences(smiles,truncating="post",
                                   padding="post",maxlen=85)
    prediction=model.predict([tf.expand_dims(protein_padded,axis=-1),
                            tf.expand_dims(smiles_padded,axis=-1)])
    return prediction

In [ ]:
prediction_or1=predictpkd(list_or1,smiles_list)
df_prect_or1 =cocodata.copy()
df_prect_or1["affinity"] = prediction_or1
df_prect_or1 = df_prect_or1.sort_values(by = 'affinity', ascending=False)
df_prect_or1.to_csv('df_prect_or1.csv')
df_top20_or1= df_prect_or1.head(20)
df_top20_or1.to_csv('df_top20_or1.csv')

In [ ]:
prediction_or2=predictpkd(list_or2,smiles_list)
df_prect_or2 =cocodata.copy()
df_prect_or2["affinity"] = prediction_or2
df_prect_or2 = df_prect_or2.sort_values(by = 'affinity', ascending=False)
df_prect_or2.to_csv('df_prect_or2.csv')
df_top20_or2= df_prect_or2.head(20)
df_top20_or2.to_csv('df_top20_or2.csv')

In [ ]:
prediction_or3=predictpkd(list_or3,smiles_list)
df_prect_or3 =cocodata.copy()
df_prect_or3["affinity"] = prediction_or3
df_prect_or3 = df_prect_or3.sort_values(by = 'affinity', ascending=False)
df_prect_or3.to_csv('df_prect_or3.csv')
df_top20_or3= df_prect_or3.head(20)
df_top20_or3.to_csv('df_top20_or3.csv')


In [ ]:
prediction_or4=predictpkd(list_or4,smiles_list)
df_prect_or4 =cocodata.copy()
df_prect_or4["affinity"] = prediction_or4
df_prect_or4 = df_prect_or4.sort_values(by = 'affinity', ascending=False)
df_prect_or4.to_csv('df_prect_or4.csv')
df_top20_or4= df_prect_or4.head(20)
df_top20_or4.to_csv('df_top20_or4.csv')


**For docking**

In [ ]:
common_id = list(reduce(set.intersection, map(set, [df_top20_or1.coconut_id, df_top20_or2.coconut_id,
                                          df_top20_or3.coconut_id,df_top20_or4.coconut_id])))

df_common =cocodata.loc[cocodata["coconut_id"].isin(common_id)]
df_common.to_csv('mol_common_to_all_4.csv')